In [2]:
# 进行环境安装准备
!pip install "numpy<=1.19.5" -i https://mirror.baidu.com/pypi/simple
!pip install "paddlex==2.0.0" -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 14.8MB 10.2MB/s eta 0:00:01
ERROR: paddlefsl 1.0.0 has requirement pillow==8.2.0, but you'll have pillow 7.1.2 which is incompatible.
ERROR: paddlefsl 1.0.0 has requirement requests~=2.24.0, but you'll have requests 2.22.0 which is incompatible.
  Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 952kB 13.0MB/s eta 0:00:01
     |████████████████████████████████| 296kB 20.9MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 15.8MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 14.9MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 13.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 52.9MB/s eta 0:00:01
     |████████████████████████████████| 112kB 22.9MB/s eta 0:00:01
     |██████

In [2]:
# 解压相对应的数据集
! unzip work/train.zip -d ./work/
! rm -f -r work/__MACOSX
! unzip work/test.zip -d ./work/
! rm -f -r work/__MACOSX

Archive:  work/train.zip
   creating: ./work/train/
  inflating: ./work/train/.DS_Store  
   creating: ./work/__MACOSX/
   creating: ./work/__MACOSX/train/
  inflating: ./work/__MACOSX/train/._.DS_Store  
   creating: ./work/train/IMAGES/
  inflating: ./work/train/IMAGES/63.jpg  
   creating: ./work/__MACOSX/train/IMAGES/
  inflating: ./work/__MACOSX/train/IMAGES/._63.jpg  
  inflating: ./work/train/IMAGES/823.jpg  
  inflating: ./work/__MACOSX/train/IMAGES/._823.jpg  
  inflating: ./work/train/IMAGES/189.jpg  
  inflating: ./work/__MACOSX/train/IMAGES/._189.jpg  
  inflating: ./work/train/IMAGES/77.jpg  
  inflating: ./work/__MACOSX/train/IMAGES/._77.jpg  
  inflating: ./work/train/IMAGES/837.jpg  
  inflating: ./work/__MACOSX/train/IMAGES/._837.jpg  
  inflating: ./work/train/IMAGES/1353.jpg  
  inflating: ./work/__MACOSX/train/IMAGES/._1353.jpg  
  inflating: ./work/train/IMAGES/1347.jpg  
  inflating: ./work/__MACOSX/train/IMAGES/._1347.jpg  
  inflating: ./work/train/IMAGES/638.jp

In [4]:
# 进行相应数据集文件的构建

import os
import random

# 遍历训练集
name = [name for name in os.listdir('work/train/IMAGES') if name.endswith('.jpg')]  # 返回指定路径下的文件和文件夹列表

train_name_list = []

for i in name:
    tmp = os.path.splitext(i)
    train_name_list.append(tmp[0])

# 构造图片-xml的链接文件ori_train.txt
with open("./work/train/ori_train.txt", "w") as f:
    for i in range(len(train_name_list)):
        if i != 0:
            f.write('\n')
        line = 'IMAGES/' + train_name_list[i] + '.jpg' + " " + "ANNOTATIONS/" + train_name_list[i] + '.xml'
        f.write(line)

# 构造label.txt
labels = ['crazing', 'inclusion', 'pitted_surface', 'scratches', 'patches', 'rolled-in_scale']

with open("./work/train/labels.txt", "w") as f:
    for i in range(len(labels)):
        line = labels[i] + '\n'
        f.write(line)

# 将ori_train.txt随机按照eval_percent分为验证集文件和训练集文件

eval_percent = 0.1  # eval_percent 验证集所占的百分比

data = []
with open("work/train/ori_train.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n')  # 使用strip()函数去掉每行的\n
        data.append(line)

index = list(range(len(data)))
random.shuffle(index)   # 将index序列的所有元素随机排序

# 构造验证集文件 val_list
cut_point = int(eval_percent * len(data))  # 验证集文件为20%数据集

with open("./work/train/val_list.txt", "w") as f:
    for i in range(cut_point):
        if i != 0:
            f.write('\n')
        line = data[index[i]]
        f.write(line)

# 构造训练集文件 train_list
with open("./work/train/train_list.txt", "w") as f:
    for i in range(cut_point, len(data)):
        if i != cut_point:
            f.write('\n')
        line = data[index[i]]
        f.write(line)

print("文件构建成功")

文件构建成功


In [5]:
# 进行数据集强化处理与数据路径配置

from paddlex import transforms as T

train_transforms = T.Compose([
    T.RandomDistort(),
    T.RandomBlur(),
    T.RandomExpand(im_padding_value=[123.675, 116.28, 103.53]), 
    T.RandomCrop(),
    T.RandomHorizontalFlip(), 
    T.BatchRandomResize(
    target_sizes=[320, 352, 384, 416, 448, 480, 512, 544, 576, 608, 640, 672, 704,736, 768],
    interp='RANDOM'), 
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eval_transforms = T.Compose([
    T.Resize(target_size=608, interp='CUBIC'), 
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 要把训练集等的配置路径全部换成我们之前定义好的路径
import paddlex as pdx

train_dataset = pdx.datasets.VOCDetection(
    data_dir='work/train',
    file_list='work/train/train_list.txt',
    label_list='work/train/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='work/train',
    file_list='work/train/val_list.txt',
    label_list='work/train/labels.txt',
    transforms=eval_transforms)


print("数据路径配置完成")

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:136: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[12-08 22:17:10 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.0. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: Depreca

2021-12-08 22:17:13 [INFO]	Starting to read file list from dataset...
2021-12-08 22:17:15 [INFO]	1260 samples in file work/train/train_list.txt, including 1260 positive samples and 0 negative samples.
creating index...
index created!
2021-12-08 22:17:15 [INFO]	Starting to read file list from dataset...
2021-12-08 22:17:15 [INFO]	140 samples in file work/train/val_list.txt, including 140 positive samples and 0 negative samples.
creating index...
index created!
数据路径配置完成


In [6]:
# 使用gpu0等

import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

In [9]:
# 模型选择与相关策略定义

import paddle

num_classes = len(train_dataset.labels) + 1
model = pdx.det.FasterRCNN(num_classes=num_classes, backbone='ResNet101_vd',with_dcn=True)

# 自定义优化器：使用CosineAnnealingDecay
train_batch_size = 8
num_steps_each_epoch = len(train_dataset) // train_batch_size
num_epochs = 160

scheduler = paddle.optimizer.lr.CosineAnnealingDecay(
    learning_rate=0.001, T_max=num_steps_each_epoch * num_epochs)

warmup_epoch = 5
warmup_steps = warmup_epoch * num_steps_each_epoch
print(warmup_steps)

scheduler = paddle.optimizer.lr.LinearWarmup(
    learning_rate=scheduler,
    warmup_steps=warmup_steps,
    start_lr=0.0,
    end_lr=0.001)

custom_optimizer = paddle.optimizer.Lamb(
        learning_rate=scheduler, 
        lamb_weight_decay=0.01,
        beta1=0.9, beta2=0.999, 
        epsilon=1e-06,
        parameters=model.net.parameters(),
        grad_clip=None, name=None)



In [ ]:
# 开始训练
# API说明：https://github.com/PaddlePaddle/PaddleX/blob/95c53dec89ab0f3769330fa445c6d9213986ca5f/paddlex/cv/models/classifier.py#L153
# 各参数介绍与调整说明：https://paddlex.readthedocs.io/zh_CN/develop/appendix/parameters.html

model.train(
    num_epochs=num_epochs,

    train_dataset=train_dataset,
    train_batch_size=train_batch_size,
    eval_dataset=eval_dataset,

    optimizer=custom_optimizer,
    
    pretrain_weights='COCO',
    save_dir='output/FasterRCNN_ResNet101_vd',
    save_interval_epochs=5,
    use_vdl=True)

2021-12-08 22:19:23 [INFO]	Downloading faster_rcnn_r101_vd_fpn_1x_coco.pdparams from https://paddledet.bj.bcebos.com/models/faster_rcnn_r101_vd_fpn_1x_coco.pdparams


100%|██████████| 360242/360242 [00:05<00:00, 62023.63KB/s]


2021-12-08 22:19:29 [INFO]	Loading pretrained model from output/FasterRCNN_ResNet101_vd/pretrain/faster_rcnn_r101_vd_fpn_1x_coco.pdparams
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3a.branch2b.conv_offset.weight is not in pretrained model
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3a.branch2b.conv_offset.bias is not in pretrained model
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3b.branch2b.conv_offset.weight is not in pretrained model
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3b.branch2b.conv_offset.bias is not in pretrained model
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3c.branch2b.conv_offset.weight is not in pretrained model
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3c.branch2b.conv_offset.bias is not in pretrained model
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3d.branch2b.conv_offset.weight is not in pretrained model
2021-12-08 22:19:31 [WARNING]	backbone.res3.res3d.branch2b.conv_offset.bias is not in pretrained model
2021-12-08 22:19:31 [WARNING]	

In [3]:
# 进行预测

import paddlex as pdx
import os
import pandas as pd
import numpy as np


name = [name for name in os.listdir('work/test/IMAGES') if name.endswith('.jpg')]

test_name_list=[]
for i in name:
    tmp = os.path.splitext(i)
    test_name_list.append(tmp[0])

#加载模型
model = pdx.load_model('output/FasterRCNN_ResNet101_vd/best_model')

# 建立一个标号和题目要求的id的映射
num2index={'crazing':0,'inclusion':1,'pitted_surface':2,'scratches':3,'patches':4,'rolled-in_scale':5}


result_list = []
# 将置信度较好的框写入result_list
for index in test_name_list:
    image_name = 'work/test/IMAGES/'+index+'.jpg'
    predicts = model.predict(image_name)
    for predict in predicts:

        # if predict['score']<0.5: continue;

        # 将bbox转化为题目中要求的格式
        tmp=predict['bbox']
        tmp[2]+=tmp[0]
        tmp[3]+=tmp[1]
        line=[index,tmp,num2index[predict['category']],predict['score']]
        result_list.append(line)

print("测试完成")


# 将结果写入目标文件,将上述result_list转化为csv

result_array = np.array(result_list)

df = pd.DataFrame(result_array, columns=['image_id', 'bbox', 'category_id', 'confidence'])
# DataFrame()函数的参数index的值相当于行索引，若不手动赋值，将默认从0开始分配。columns的值相当于列索引，若不手动赋值，也将默认从0开始分配。

df.to_csv('submission-01.csv', index=None)
print("写入完成")

# 读取文件数据
df = pd.read_csv('./submission-01.csv')
# 按照列值排序
data = df.sort_values(by="image_id", ascending=True)
# 把新的数据写入文件
data.to_csv('submission-1.csv', mode='a+', index=False)

2021-12-09 09:03:48 [INFO]	Model[FasterRCNN] loaded.
测试完成


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


写入完成


In [1]:
# 结果可视化,PaddleX提供了一系列模型预测和结果分析的可视化函数

import paddlex as pdx
model = pdx.load_model('output/FasterRCNN_ResNet101_vd/best_model')
index = '1402'
image_name = 'work/test/IMAGES/'+index+'.jpg'
predicts = model.predict(image_name)
pdx.det.visualize(image_name, predicts, threshold=0.5, save_dir='./data')

ModuleNotFoundError: No module named 'paddlex'

In [7]:
#进行结果处理

import pandas as pd

filename='submission-1.csv'

df = pd.read_csv(filename)

data = data[(df['confidence'] >= 0.75)]   #筛选置信度      

# 按照列值排序
data = data.sort_values(by="image_id", ascending=True)

data.to_csv('置信度高文件-1.csv', mode='a+', index=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [ ]:
#进行结果处理


import pandas as pd

filename='submission-1.csv'

df = pd.read_csv(filename)

data = df[(df['confidence'] <= 0.75)] 
data = data[(df['confidence'] >= 0.5)]   #筛选置信度小于规定值      

# 按照列值排序
data = data.sort_values(by="category_id", ascending=True)

data.to_csv('置信度低文件-1.csv', mode='a+', index=False)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 